#### Count distribution of episode, patient in all the episodes, first episode, last episode

##### In all episodes (1)

In [1]:
## In all episodes before encoding of F90
import pandas as pd
import ast

trajectory_df = pd.read_csv(
    "/mnt/work/workbench/dipendrp/Paper3/Data/Trajectory_Full_ICD10_ATC_Cluster3Label.csv"
)


# Function to check the presence of diagnosis codes starting with 'F90'
def check_F90(diagnosis):
    if pd.isna(diagnosis):
        return "NaN Only"
    diagnosis_list = ast.literal_eval(diagnosis)
    # Check if any diagnosis code starts with 'F90'
    if any(code.startswith("F90") for code in diagnosis_list):
        return "F90 Present"
    return "Other Diagnosis"


# Apply the function to the 'diagnosis' column
trajectory_df["diagnosis_check"] = trajectory_df["diagnosis"].apply(check_F90)

# Count the occurrences of each type of diagnosis for each patient
diagnosis_counts = (
    trajectory_df.groupby(["pasient", "diagnosis_check"]).size().unstack(fill_value=0)
)

# Display the counts
display(trajectory_df)

# Calculate and display the number of patients with each diagnosis type
patients_summary = diagnosis_counts.gt(0).sum()
print("Number of patients with each diagnosis type:")
print(patients_summary)

# Calculate and display the number of episodes with each diagnosis type
episodes_summary = diagnosis_counts.sum()
print("\nNumber of episodes with each diagnosis type:")
print(episodes_summary)

,pasient,episode_id,episode_start_date,episode_end_date,gender,age_group,diagnosis,actual_med_Full_ATC,Length_of_Episode,Count_visit,Therapy_ratio,tillnextepisode,cluster,cluster_distances,patient_age,age,diagnosis_check
0,2,3836.0,2013-07-02,2015-07-03,F,Teenager,"['F930', 'F431']",NaN,732,49,0.500,NaN,1,3.075655,12,12.238444,Other Diagnosis
1,4,25641.0,2015-09-28,2017-10-04,M,Teenager,"['F429', 'F422']",NaN,738,160,0.612,NaN,1,3.245776,13,13.738817,Other Diagnosis
2,6,24660.0,2004-10-01,2007-07-06,M,MiddleChildhood,NaN,NaN,1009,98,0.500,NaN,1,3.218743,6,6.004230,NaN Only
3,7,22380.0,2015-09-28,2016-10-06,M,MiddleChildhood,['H932'],NaN,375,80,0.293,NaN,1,2.978143,8,8.739399,Other Diagnosis
4,8,28188.0,2002-12-30,2005-01-09,F,Teenager,['F321'],NaN,742,52,0.553,NaN,1,3.281965,16,16.983237,Other Diagnosis
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22671,22553,7950.0,2007-01-01,2010-07-06,F,Teenager,"['E66', 'E669', 'F331', 'F432', 'F332', 'F6030...",NaN,1283,390,0.523,NaN,2,7.030308,15,15.250142,Other Diagnosis
22672,22595,16793.0,2010-10-05,2011-10-05,F,Teenager,NaN,NaN,366,18,0.556,NaN,1,3.524008,15,15.742965,NaN Only
22673,22603,18636.0,2005-10-04,2008-10-01,F,Teenager,NaN,NaN,1094,522,0.428,NaN,0,7.937477,15,15.740227,NaN Only
22674,22604,7521.0,2002-12-30,2003-04-06,M,Teenager,NaN,NaN,98,31,0.414,NaN,0,7.191046,12,12.736743,NaN Only


Number of patients with each diagnosis type:
diagnosis_check
F90 Present        4411
NaN Only           7334
Other Diagnosis    8996
dtype: int64

Number of episodes with each diagnosis type:
diagnosis_check
F90 Present        5005
NaN Only           7833
Other Diagnosis    9838
dtype: int64


In [2]:
## In all episodes before encoding of F90
import pandas as pd
import ast

trajectory_df = pd.read_csv(
    "/mnt/work/workbench/dipendrp/Paper3/Data/Trajectory_Full_ICD10_ATC_Cluster3Label.csv"
)

patient_trajectory_df = trajectory_df
print(f"Total Episodes: {trajectory_df['episode_id'].nunique()}")
print(f"Episode with NaN: {trajectory_df['diagnosis'].isna().sum()}")
print(
    f"Episode with F90: {trajectory_df[trajectory_df['diagnosis'].str.contains('F90', na=False)]['episode_id'].nunique()}"
)
# count Episode without F90, excluding NaN
trajectory_df = trajectory_df[~trajectory_df["diagnosis"].isna()]
print(
    f"Episode without F90, and NaN: {trajectory_df[~trajectory_df['diagnosis'].str.contains('F90', na=False)]['episode_id'].nunique()}"
)

print("\n")
totalPatients = patient_trajectory_df["pasient"].nunique()
print(f"Total Patients: {totalPatients}")
patientWithNaN = patient_trajectory_df[patient_trajectory_df["diagnosis"].isna()][
    "pasient"
].nunique()
print(f"Patients with NaN: {patientWithNaN}")
patient_trajectory_df = trajectory_df[~trajectory_df["diagnosis"].isna()]

patientWithF90 = patient_trajectory_df[
    patient_trajectory_df["diagnosis"].str.contains("F90", na=False)
]["pasient"].nunique()
print(f"Patients with F90: {patientWithF90}")
print(
    f"Total Patient excluding NaN and F90: {totalPatients - patientWithNaN - patientWithF90}"
)

Total Episodes: 22676
Episode with NaN: 7833
Episode with F90: 5005
Episode without F90, and NaN: 9838


Total Patients: 19248
Patients with NaN: 7334
Patients with F90: 4411
Total Patient excluding NaN and F90: 7503


In [3]:
## In all episodes before encoding of F90
import pandas as pd
import ast

trajectory_df = pd.read_csv(
    "/mnt/work/workbench/dipendrp/Paper3/Data/Trajectory_Full_ICD10_ATC_Cluster3Label.csv"
)
patient_trajectory_df = trajectory_df
print(f"Total Episodes: {trajectory_df['episode_id'].nunique()}")
print(f"Episode with NaN: {trajectory_df['diagnosis'].isna().sum()}")
print(
    f"Episode with F90: {trajectory_df[trajectory_df['diagnosis'].str.contains('F90', na=False)]['episode_id'].nunique()}"
)
# count Episode without F90, excluding NaN
trajectory_df = trajectory_df[~trajectory_df["diagnosis"].isna()]
print(
    f"Episode without F90, and NaN: {trajectory_df[~trajectory_df['diagnosis'].str.contains('F90', na=False)]['episode_id'].nunique()}"
)

print("\n")

print(f"Total Patients: {patient_trajectory_df['pasient'].nunique()}")
print(
    f"Patients with NaN: {patient_trajectory_df[patient_trajectory_df['diagnosis'].isna()]['pasient'].nunique()}"
)

print(
    f"Patients with F90: {patient_trajectory_df[patient_trajectory_df['diagnosis'].str.contains('F90', na=False)]['pasient'].nunique()}"
)
print(
    f"Total Patient excluding NaN and F90: {patient_trajectory_df['pasient'].nunique() - (patient_trajectory_df[patient_trajectory_df['diagnosis'].str.contains('F90', na=False)]['pasient'].nunique()) - patient_trajectory_df[patient_trajectory_df['diagnosis'].isna()]['pasient'].nunique()}"
)

Total Episodes: 22676
Episode with NaN: 7833
Episode with F90: 5005
Episode without F90, and NaN: 9838


Total Patients: 19248
Patients with NaN: 7334
Patients with F90: 4411
Total Patient excluding NaN and F90: 7503


In [4]:
## In all episodes after encoding of F90
import pandas as pd
import ast

trajectory_df = pd.read_csv(
    "/mnt/work/workbench/dipendrp/Paper3/Data/Trajectory_Full_ICD10_ATC_Cluster3Label.csv"
)

forPatient_Trajectory_df = trajectory_df

# Count the total episodes, episode with NaN
print(f"Total Episodes: {trajectory_df['episode_id'].nunique()}")
episodeNaNCount = trajectory_df["diagnosis"].isna().sum()
print(f"Episode with NaN: {episodeNaNCount}")

noNanTrajectory_df = trajectory_df[~trajectory_df["diagnosis"].isna()]
print(f"Total Episodes without NaN: {noNanTrajectory_df['episode_id'].nunique()}")

noNanTrajectory_df["ADHD_NoADHD"] = noNanTrajectory_df["diagnosis"].apply(
    lambda x: 1 if "F90" in str(x) else 0
)

print(
    f"Episode with ADHD, excluding NaN: {noNanTrajectory_df[noNanTrajectory_df['ADHD_NoADHD'] == 1]['episode_id'].nunique()}"
)
print(
    f"Episode without ADHD, excluding NaN: {noNanTrajectory_df[noNanTrajectory_df['ADHD_NoADHD'] == 0]['episode_id'].nunique()}"
)
print("\n")
# Display the count of patient with ADHD and without ADHD
print(f"Total Patient including NaN: {forPatient_Trajectory_df['pasient'].nunique()}")

# Get unique patients
unique_patients = noNanTrajectory_df["pasient"].unique()
print(f"Total Patient excluding NaN: {len(unique_patients)}")

print(
    f"Total Patient with NaN: {forPatient_Trajectory_df[forPatient_Trajectory_df['diagnosis'].isna()]['pasient'].nunique()}"
)

# Get unique patients with ADHD_NoADHD == 1
adhd_patients = noNanTrajectory_df[noNanTrajectory_df["ADHD_NoADHD"] == 1][
    "pasient"
].unique()
print(f"Total Patient, excluding NaN with F90: {len(adhd_patients)}")

print(
    f"Total Patient excluding NaN and F90: {forPatient_Trajectory_df['pasient'].nunique() - (forPatient_Trajectory_df[forPatient_Trajectory_df['diagnosis'].str.contains('F90', na=False)]['pasient'].nunique()) - forPatient_Trajectory_df[forPatient_Trajectory_df['diagnosis'].isna()]['pasient'].nunique()}"
)

Total Episodes: 22676
Episode with NaN: 7833
Total Episodes without NaN: 14843
Episode with ADHD, excluding NaN: 5005
Episode without ADHD, excluding NaN: 9838


Total Patient including NaN: 19248
Total Patient excluding NaN: 13142
Total Patient with NaN: 7334
Total Patient, excluding NaN with F90: 4411
Total Patient excluding NaN and F90: 7503


/mnt/scratch/tmp/ipykernel_26443/2462513790.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  noNanTrajectory_df["ADHD_NoADHD"] = noNanTrajectory_df["diagnosis"].apply(


##### In first episodes (1)

In [5]:
import pandas as pd
import ast

trajectory_df = pd.read_csv(
    "/mnt/work/workbench/dipendrp/Paper3/Data/Trajectory_Full_ICD10_ATC_Cluster3Label.csv"
)
first_episode_df = (
    trajectory_df.sort_values(by=["pasient", "episode_start_date"], ascending=True)
    .groupby("pasient")
    .nth(0)
)
display(first_episode_df)
print(f"Total Patient including NaN: {first_episode_df['pasient'].nunique()}")
episodeNaNCount = first_episode_df["diagnosis"].isna().sum()
print(f"Patients with NaN: {episodeNaNCount}")
print(
    f"Patients with F90: {first_episode_df[first_episode_df['diagnosis'].str.contains('F90', na=False)]['episode_id'].nunique()}"
)
print(
    f"Patients without F90, include NaN: {first_episode_df[~first_episode_df['diagnosis'].str.contains('F90', na=False)]['episode_id'].nunique()}"
)

first_episode_df = first_episode_df[~first_episode_df["diagnosis"].isna()]
print(
    f"Patients without F90, and NaN: {first_episode_df[~first_episode_df['diagnosis'].str.contains('F90', na=False)]['episode_id'].nunique()}"
)

,pasient,episode_id,episode_start_date,episode_end_date,gender,age_group,diagnosis,actual_med_Full_ATC,Length_of_Episode,Count_visit,Therapy_ratio,tillnextepisode,cluster,cluster_distances,patient_age,age
20938,1,15745.0,2005-01-03,2017-07-06,M,Teenager,['F432'],NaN,4568,232,0.361,NaN,0,9.395895,13,13.495144
0,2,3836.0,2013-07-02,2015-07-03,F,Teenager,"['F930', 'F431']",NaN,732,49,0.500,NaN,1,3.075655,12,12.238444
1,4,25641.0,2015-09-28,2017-10-04,M,Teenager,"['F429', 'F422']",NaN,738,160,0.612,NaN,1,3.245776,13,13.738817
2,6,24660.0,2004-10-01,2007-07-06,M,MiddleChildhood,NaN,NaN,1009,98,0.500,NaN,1,3.218743,6,6.004230
3,7,22380.0,2015-09-28,2016-10-06,M,MiddleChildhood,['H932'],NaN,375,80,0.293,NaN,1,2.978143,8,8.739399
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20931,22638,28365.0,2013-12-31,2016-04-05,M,MiddleChildhood,"['H520', 'F900']",NaN,827,62,0.246,NaN,1,3.229090,10,10.751761
20932,22639,4569.0,2012-10-01,2013-04-04,M,Teenager,NaN,NaN,186,42,0.500,NaN,1,3.793842,16,16.736826
20933,22640,28667.0,2008-06-30,2008-07-03,M,Teenager,['F845'],NaN,4,6,0.000,NaN,1,4.574324,15,15.230977
20934,22641,1747.0,2009-09-28,2010-01-07,F,Teenager,"['G409', 'F438']",NaN,102,33,0.323,634.0,1,4.841614,14,14.732678


Total Patient including NaN: 19248
Patients with NaN: 6871
Patients with F90: 3916
Patients without F90, include NaN: 15332
Patients without F90, and NaN: 8461


##### In last episodes (1)

In [6]:
import pandas as pd
import ast

trajectory_df = pd.read_csv(
    "/mnt/work/workbench/dipendrp/Paper3/Data/Trajectory_Full_ICD10_ATC_Cluster3Label.csv"
)
first_episode_df = (
    trajectory_df.sort_values(by=["pasient", "episode_start_date"], ascending=True)
    .groupby("pasient")
    .nth(-1)
)
# From trajectory_df select only those rows which are the first episode of the each patient and save it to a new dataframe
last_episode_df = (
    trajectory_df.sort_values(by=["pasient", "episode_start_date"], ascending=True)
    .groupby("pasient")
    .nth(-1)
)
display(last_episode_df)
print(f"Total Patient including NaN: {last_episode_df['pasient'].nunique()}")
episodeNaNCount = last_episode_df["diagnosis"].isna().sum()
print(f"Patients with NaN: {episodeNaNCount}")
print(
    f"Patients with F90: {last_episode_df[last_episode_df['diagnosis'].str.contains('F90', na=False)]['episode_id'].nunique()}"
)
print(
    f"Patients without F90, include NaN: {last_episode_df[~last_episode_df['diagnosis'].str.contains('F90', na=False)]['episode_id'].nunique()}"
)

last_episode_df = last_episode_df[~last_episode_df["diagnosis"].isna()]
print(
    f"Patients without F90, and NaN: {last_episode_df[~last_episode_df['diagnosis'].str.contains('F90', na=False)]['episode_id'].nunique()}"
)

,pasient,episode_id,episode_start_date,episode_end_date,gender,age_group,diagnosis,actual_med_Full_ATC,Length_of_Episode,Count_visit,Therapy_ratio,tillnextepisode,cluster,cluster_distances,patient_age,age
20938,1,15745.0,2005-01-03,2017-07-06,M,Teenager,['F432'],NaN,4568,232,0.361,NaN,0,9.395895,13,13.495144
0,2,3836.0,2013-07-02,2015-07-03,F,Teenager,"['F930', 'F431']",NaN,732,49,0.500,NaN,1,3.075655,12,12.238444
1,4,25641.0,2015-09-28,2017-10-04,M,Teenager,"['F429', 'F422']",NaN,738,160,0.612,NaN,1,3.245776,13,13.738817
2,6,24660.0,2004-10-01,2007-07-06,M,MiddleChildhood,NaN,NaN,1009,98,0.500,NaN,1,3.218743,6,6.004230
3,7,22380.0,2015-09-28,2016-10-06,M,MiddleChildhood,['H932'],NaN,375,80,0.293,NaN,1,2.978143,8,8.739399
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20931,22638,28365.0,2013-12-31,2016-04-05,M,MiddleChildhood,"['H520', 'F900']",NaN,827,62,0.246,NaN,1,3.229090,10,10.751761
20932,22639,4569.0,2012-10-01,2013-04-04,M,Teenager,NaN,NaN,186,42,0.500,NaN,1,3.793842,16,16.736826
20933,22640,28667.0,2008-06-30,2008-07-03,M,Teenager,['F845'],NaN,4,6,0.000,NaN,1,4.574324,15,15.230977
20935,22641,16792.0,2011-10-03,2013-12-31,F,Teenager,"['G409', 'F900']",['N06BA04'],821,127,0.504,NaN,2,3.050173,16,16.745039


Total Patient including NaN: 19248
Patients with NaN: 6553
Patients with F90: 4236
Patients without F90, include NaN: 15012
Patients without F90, and NaN: 8459


### Create a column ADHD_NoADHD, encode it as 1 if any of the episodes diagnosis of the patient contains F90, if it contains all NaN in all the episode of patient encode it as 2 , but if it does not contains any F90, neither it has NaN in all the episodes, then encode it as 3 

In [7]:
import pandas as pd
import ast

trajectory_df = pd.read_csv(
    "/mnt/work/workbench/dipendrp/Paper3/Data/Trajectory_Full_ICD10_ATC_Cluster3Label.csv"
)


# Function to determine the encoding for ADHD_NoADHD
def encode_adhd(diagnoses):
    # Check if all diagnoses are NaN
    if all(pd.isna(d) for d in diagnoses):
        return 2
    # Check if any diagnosis contains 'F90'
    elif any("F90" in str(d) for d in diagnoses):
        return 1
    # If neither of the above, return 3
    else:
        return 3


# Group by 'pasient' and apply the encoding function to the 'diagnosis' column
trajectory_df["ADHD_NoADHD"] = trajectory_df.groupby("pasient")["diagnosis"].transform(
    encode_adhd
)

# Display the resulting dataframe
display(trajectory_df)

# Count total unique patients with 3 in ADHD_NoADHD
print(f"Total Patients: {trajectory_df['pasient'].nunique()}")
print(
    f"Patients with ADHD: {trajectory_df[trajectory_df['ADHD_NoADHD'] == 1]['pasient'].nunique()}"
)
print(
    f"Patients without ADHD: {trajectory_df[trajectory_df['ADHD_NoADHD'] == 3]['pasient'].nunique()}"
)
print(
    f"Patients with NaN: {trajectory_df[trajectory_df['ADHD_NoADHD'] == 2]['pasient'].nunique()}"
)

,pasient,episode_id,episode_start_date,episode_end_date,gender,age_group,diagnosis,actual_med_Full_ATC,Length_of_Episode,Count_visit,Therapy_ratio,tillnextepisode,cluster,cluster_distances,patient_age,age,ADHD_NoADHD
0,2,3836.0,2013-07-02,2015-07-03,F,Teenager,"['F930', 'F431']",NaN,732,49,0.500,NaN,1,3.075655,12,12.238444,3
1,4,25641.0,2015-09-28,2017-10-04,M,Teenager,"['F429', 'F422']",NaN,738,160,0.612,NaN,1,3.245776,13,13.738817,3
2,6,24660.0,2004-10-01,2007-07-06,M,MiddleChildhood,NaN,NaN,1009,98,0.500,NaN,1,3.218743,6,6.004230,2
3,7,22380.0,2015-09-28,2016-10-06,M,MiddleChildhood,['H932'],NaN,375,80,0.293,NaN,1,2.978143,8,8.739399,3
4,8,28188.0,2002-12-30,2005-01-09,F,Teenager,['F321'],NaN,742,52,0.553,NaN,1,3.281965,16,16.983237,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22671,22553,7950.0,2007-01-01,2010-07-06,F,Teenager,"['E66', 'E669', 'F331', 'F432', 'F332', 'F6030...",NaN,1283,390,0.523,NaN,2,7.030308,15,15.250142,3
22672,22595,16793.0,2010-10-05,2011-10-05,F,Teenager,NaN,NaN,366,18,0.556,NaN,1,3.524008,15,15.742965,2
22673,22603,18636.0,2005-10-04,2008-10-01,F,Teenager,NaN,NaN,1094,522,0.428,NaN,0,7.937477,15,15.740227,2
22674,22604,7521.0,2002-12-30,2003-04-06,M,Teenager,NaN,NaN,98,31,0.414,NaN,0,7.191046,12,12.736743,2


Total Patients: 19248
Patients with ADHD: 4411
Patients without ADHD: 8731
Patients with NaN: 6106


In [8]:
import pandas as pd
import numpy as np

# Your existing dataframe code
data = {
    "pasient": [2, 2, 2, 8, 9, 9, 13, 15, 16, 17, 17],
    "episode_id": [
        3836,
        25641,
        22380,
        28188,
        17749,
        29685,
        23747,
        7847,
        16500,
        9632,
        9635,
    ],
    "episode_start_date": [
        "2010-07-02",
        "2015-09-28",
        "2015-09-28",
        "2002-12-30",
        "2016-07-05",
        "2009-03-30",
        "2003-09-30",
        "2009-06-30",
        "2014-12-30",
        "2010-01-02",
        "2010-01-06",
    ],
    "diagnosis": [
        np.nan,
        "['F429', 'F422']",
        "['F429', 'F422']",
        "['F321']",
        "['G430', 'F4322', 'G431']",
        "['F845']",
        np.nan,
        "['F500']",
        np.nan,
        np.nan,
        "['J459', 'F901']",
    ],
}

trajectory_df = pd.DataFrame(data)


# Function to determine the encoding for ADHD_NoADHD
def encode_adhd(diagnoses):
    # Check if all diagnoses are NaN
    if all(pd.isna(d) for d in diagnoses):
        return 2
    # Check if any diagnosis contains 'F90'
    elif any("F90" in str(d) for d in diagnoses):
        return 1
    # If neither of the above, return 3
    else:
        return 3


# Group by 'pasient' and apply the encoding function to the 'diagnosis' column
trajectory_df["ADHD_NoADHD"] = trajectory_df.groupby("pasient")["diagnosis"].transform(
    encode_adhd
)

# Display the resulting dataframe
display(trajectory_df)

# Count total unique patients with 3 in ADHD_NoADHD
print(f"Total Patients: {trajectory_df['pasient'].nunique()}")
print(
    f"Patients with ADHD: {trajectory_df[trajectory_df['ADHD_NoADHD'] == 1]['pasient'].nunique()}"
)
print(
    f"Patients without ADHD: {trajectory_df[trajectory_df['ADHD_NoADHD'] == 3]['pasient'].nunique()}"
)
print(
    f"Patients with NaN: {trajectory_df[trajectory_df['ADHD_NoADHD'] == 2]['pasient'].nunique()}"
)

,pasient,episode_id,episode_start_date,diagnosis,ADHD_NoADHD
0,2,3836,2010-07-02,NaN,3
1,2,25641,2015-09-28,"['F429', 'F422']",3
2,2,22380,2015-09-28,"['F429', 'F422']",3
3,8,28188,2002-12-30,['F321'],3
4,9,17749,2016-07-05,"['G430', 'F4322', 'G431']",3
5,9,29685,2009-03-30,['F845'],3
6,13,23747,2003-09-30,NaN,2
7,15,7847,2009-06-30,['F500'],3
8,16,16500,2014-12-30,NaN,2
9,17,9632,2010-01-02,NaN,1


Total Patients: 7
Patients with ADHD: 1
Patients without ADHD: 4
Patients with NaN: 2
